<a href="https://colab.research.google.com/github/nicolaiberk/Imbalanced/blob/master/01_IntroSML_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An Introduction to Supervised Learning with Scikit learn

Classifications can take many forms. Today, we will train a simple binary sentiment classifier, using a subset of 10,000 [Amazon Reviews provided as part of a Kaggle competition](https://www.kaggle.com/datasets/bittlingmayer/amazonreviews?resource=download). We will use the amazing `scikit-learn` package to transform the data and train our classifier.

## A Minimal Example

We first load some basic packages and the data:

In [1]:
# fundamental packages
import numpy as np
import pandas as pd

# load some data to train our classifier on
reviews = pd.read_csv("https://www.dropbox.com/s/zup4rcr8j5jz0wr/sub_sample.csv?dl=1")
reviews["bin_label"] = reviews.label == "good"


In [2]:
reviews.shape

(10000, 3)

In [3]:
reviews.head()

,label,text,bin_label
0,good,District 9 Blu-Ray Edition: This Blu-Ray disc ...,True
1,bad,Great for it's age I guess...: This book smell...,False
2,bad,OOps !.. too much religious stuff here: I gues...,False
3,good,A Great Surprise!: When I first heard about th...,True
4,bad,Useless: Worthless for med school admission. H...,False


The data has a simple structure, with 10,000 observations and two variables/columns, "label" and "text". The label is either "good"or "bad". We added a binary version of the label as a third variable. Our task is now to train a classifier that cann tell tehse two labels apart, based on the text of the review. For that, we need some tools!

### A Quick Intro to `scikit-learn`

In [4]:
import IPython
url = 'https://scikit-learn.org/stable/'
iframe = '<iframe src=' + url + ' width=1600 height=350></iframe>'
IPython.display.HTML(iframe)

/usr/local/lib/python3.10/dist-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


`scikit-learn` is an amazing package, catering to pretty much every need of data scientist. **In order to train a classifier, we need a model that we can train and a vectorizer to transform our data**, that's pretty much it. `scikit-learn` offers much more (please go check it out already!), like a function to transform our data in training and testing data and functions to bind them together and produce our metrics. **We load all of this below**:

In [5]:
# load relevant tools

## A model (choose from API)
from sklearn.linear_model import LogisticRegression as LogReg

## A vectorizer to transform our text into numbers
from sklearn.feature_extraction.text import TfidfVectorizer

## A function to split our data into train and test set
from sklearn.model_selection import train_test_split

## A pipeline to put it all together, and a few functions to compute how well our classifier performs
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

Let's split our data into train and test set:

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    reviews.text, reviews.bin_label, test_size=0.33, random_state=42)

Now we need literally two lines of code to train the classifier.

In [7]:
pipe = Pipeline([('Tfidf', TfidfVectorizer()), ('LogReg', LogReg())])
pipe.fit(X_train, y_train)

Pipeline(steps=[('Tfidf', TfidfVectorizer()), ('LogReg', LogisticRegression())])

![](https://media.giphy.com/media/zXMRfbsHOAire/giphy.gif)

Don't believe me? Check for yourself:

In [8]:
pipe.predict(["This is a great movie",
              "Never hated something as much as this movie"])

array([ True, False])

It does predict our examples well, but how good is the accuracy?

In [9]:
y_pred = pipe.predict(X_test)
pd.crosstab(y_test, y_pred)

col_0,False,True
bin_label,,
False,1434,208
True,235,1423


In [10]:
## define a custom function to report metrics
def accuracy_report(y_test, y_pred):
  print("Accuracy: ",  round(accuracy_score(y_test, y_pred), 3))
  print("Recall: ",    round(recall_score(y_test, y_pred), 3))
  print("Precision: ", round(precision_score(y_test, y_pred), 3))
  print("F1: ",        round(f1_score(y_test, y_pred), 3))

accuracy_report(y_test, y_pred)

Accuracy:  0.866
Recall:  0.858
Precision:  0.872
F1:  0.865


Pretty good, huh? Let's see how this works in more detail!

## Under the Hood

Let's show this based on a very simple example. We generate a set of example texts that are positive or negative reviews and check what the classifier does:

In [11]:
example_revs = ["This is a great, great movie",
                "This is a horrible movie",
                "Waste of time",
                "Beautiful"]

example_y = [True, False, False, True]

### Vectorization

We choose a vectorizer for our text [from `scikit-learn`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text) and assign it to an object so we can fit it:

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
vec  = CountVectorizer()

Then, we fit it to our example reviews and transform the text into numbers:

In [13]:
sparse_mtrx = vec.fit_transform(example_revs)
print(vec.get_feature_names_out(), "\n", sparse_mtrx.toarray())

['beautiful' 'great' 'horrible' 'is' 'movie' 'of' 'this' 'time' 'waste'] 
 [[0 2 0 1 1 0 1 0 0]
 [0 0 1 1 1 0 1 0 0]
 [0 0 0 0 0 1 0 1 1]
 [1 0 0 0 0 0 0 0 0]]


We can see that the vectorizer simply counts the occurence of each word in each text. The vectorizer by default strips all accents and converts all words into lowercase. Now we can use the `transform()` function to transform new texts into the same format. This is particularly important when we need to transform texts in the test set into a matrix based on the training set.

In [14]:
vec.transform(["This movie is not good."]).toarray()

array([[0, 0, 0, 1, 1, 0, 1, 0, 0]])

We can see that some features ('not' and 'good') from this new text are not encoded, as the vectorizer does not have an appropriate column in the document-term-matrix.

Vectorizers have many more features that can be used to preprocess the text. Look up the arguments of the [`CountVectorizer()`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) and test it yourself:

In [15]:
vec = CountVectorizer(stop_words=["this", "is", "of"])
sparse_mtrx = vec.fit_transform(example_revs)

## use the command from above to rpint your transformed matrix
print(vec.get_feature_names_out(), "\n", sparse_mtrx.toarray())

['beautiful' 'great' 'horrible' 'movie' 'time' 'waste'] 
 [[0 2 0 1 0 0]
 [0 0 1 1 0 0]
 [0 0 0 0 1 1]
 [1 0 0 0 0 0]]


### Fitting the Model

Now that we know how to convert text into numbers, we can fit a classifier to the data in order to predict observations in the test set (we use our initial data again).

In [19]:
## train-test-split
X_train, X_test, y_train, y_test = train_test_split(
  reviews.text, reviews.bin_label, test_size=0.33, random_state=42)

## vectorize data
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(ngram_range=(1,2))

X_train = vec.fit_transform(X_train)

## load a classifier of your choosing
from sklearn.linear_model import SGDClassifier as SVM
clsfr = SVM()

## fit
clsfr.fit(X_train, y_train)

SGDClassifier()

Now we can assess performance same as before:

In [20]:
X_test  = vec.transform(X_test)
y_pred = clsfr.predict(X_test)

pd.crosstab(y_test, y_pred)

col_0,False,True
bin_label,,
False,1472,170
True,206,1452


In [21]:
accuracy_report(y_test, y_pred)

Accuracy:  0.886
Recall:  0.876
Precision:  0.895
F1:  0.885


## BONUS: Improving your Model

### Preprocessing with [`spaCy`](https://spacy.io/)

Sometimes, you might want to pre-select features based on your classification problem. For example, when you are interested in the topic of a text, it might be sufficient to assess the nouns which are used, whereas other words might introduce mostly noise. Other tasks might require you to identify the object in a sentence or the organisation mentioned in a text. `spaCy` can identify these words through **parts-of-speech tagging**, **Dependency Parsing**, and **named entity recognition**. However, `spaCy` can do much more. Their [website](https://course.spacy.io/en/) provides an entire course from finding words to training a neural network.

#### Parts-of-speech tagging

In [22]:
import spacy
nlp = spacy.load('en_core_web_sm')


# Process a text
doc = nlp("Not the hero we deserve, but the hero we need.")

# Iterate over the tokens
for token in doc:
    # Print the text and the predicted part-of-speech tag
    print(token.text, token.pos_)

# what's PRON? get an explanation:
spacy.explain("PRON")

Not PART
the DET
hero NOUN
we PRON
deserve VERB
, PUNCT
but CCONJ
the DET
hero NOUN
we PRON
need VERB
. PUNCT


'pronoun'

In [23]:
# retain only the nouns of a set of texts:
docs = ["May the force be with you.",
        "You're gonna need a bigger boat!",
        "Fly, you fools!",
        "And I will strike down upon thee with great vengeance and furious anger!",
        "You can't handle the truth!",
        "You take the blue pill, the story ends; you wake up in your bed and believe whatever you want to believe.",
        "I love the smell of napalm in the morning."]


for doc in nlp.pipe(docs):
  print([token.text for token in doc if token.pos_ == 'NOUN'])


['force']
['boat']
['fools']
['vengeance', 'anger']
['truth']
['pill', 'story', 'bed']
['smell', 'napalm', 'morning']


Another package commonly used in text analysis is [`nltk`](https://www.nltk.org/). It has similar functionalities as `spacy` (e.g. parts-of-speech-tagging) but a slightly different implementation. Below, it is shown how to remove stopwords and stem with `nltk`.

#### Stopword removal

Many words that are constantly used in everyday language are usually not very informative about the content of text (see [Pennebaker 2011](http://secretlifeofpronouns.com/) for a contrasting perspective). These words are called 'stopwords' in NLP and usually considered clutter that could and should be removed.

Note however that preprocessing can heavily affect model results ([Denny and Spirling 2018](https://www.cambridge.org/core/journals/political-analysis/article/text-preprocessing-for-unsupervised-learning-why-it-matters-when-it-misleads-and-what-to-do-about-it/AA7D4DE0AA6AB208502515AE3EC6989E)). How to preprocess text in general is a decision that should be made based on careful consideration of the problem at hand ([Grimmer and Stewart 2013](https://www.cambridge.org/core/services/aop-cambridge-core/content/view/F7AAC8B2909441603FEB25C156448F20/S1047198700013401a.pdf)).

In [24]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [25]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

filtered_docs = []

for doc in docs:
  filtered_doc = " ".join([w for w in word_tokenize(doc) if not w.lower() in stop_words])
  filtered_docs.append(filtered_doc)

filtered_docs

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['May force .',
 "'re gon na need bigger boat !",
 'Fly , fools !',
 'strike upon thee great vengeance furious anger !',
 "ca n't handle truth !",
 'take blue pill , story ends ; wake bed believe whatever want believe .',
 'love smell napalm morning .']

#### Stemming

In [26]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

stemmed_docs = []

for doc in docs:
  stemmed_doc = " ".join([stemmer.stem(w) for w in word_tokenize(doc)])
  stemmed_docs.append(stemmed_doc)

stemmed_docs

['may the forc be with you .',
 'you re gon na need a bigger boat !',
 'fli , you fool !',
 'and i will strike down upon thee with great vengeanc and furious anger !',
 "you ca n't handl the truth !",
 'you take the blue pill , the stori end ; you wake up in your bed and believ whatev you want to believ .',
 'i love the smell of napalm in the morn .']

### Optimizing Model Fit and Avoiding Overfit with CrossValidation

**Overfitting** describes the problem that we might have a classifier fitting our data a little too well, in that it **does not describe general patterns** anymore, but potentially incorporate some **idiosyncratic noise** in the training data. The visualisation below from the [`scikit-learn` guide on overfitting](https://scikit-learn.org/stable/auto_examples/model_selection/plot_underfitting_overfitting.html?highlight=crossvalidation#underfitting-vs-overfitting) tries to describe this problem.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_underfitting_overfitting_001.png)


Often, we might want to test models with different parameters and choose the one that fits best to solve our problem. This will generally increase the fit of our data but also potentially lead to overfitting. A particularly good way to deal with this is called **cross-validation**. In this process, the training data is divided into equally sized subsets and then, several classifiers are trained to predict each subset from the other subsets. This way, the influence of single observations is reduced, because each observation is in the test set once. More on this [here](https://scikit-learn.org/stable/modules/cross_validation.html) and [here](https://cssbook.net/chapter08.html#8_5_3).

`scikit-learn` contains many models like `LogisticRegressionCV()`, which implement this by default. However, using `GridSearchCV()`, we can optimise the parameters of any model. In order to optimise the parameters of our model, we first need to check which parameters *can* be tuned:

In [27]:
?LogReg()

Object `LogReg()` not found.


In `LogisticRegression()`, we can change the regularization technique with `penalty` (regularisation is a [process](https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c) decreasing the importance of unimportant features, which avoids overfitting when classifying with many features). We can also change the degree of regularization using `C`. 

In [28]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Set the parameters by cross-validation
tuned_parameters = [
    {"penalty": ["l1", "l2"], "C": [1, 10, 100, 1000]}
]

score = "f1"

clf = GridSearchCV(LogReg(solver = 'liblinear'), tuned_parameters, scoring="%s_macro" % score)
clf.fit(X_train, y_train)

print("Best parameters set found on development set:")
print(clf.best_params_)

Best parameters set found on development set:
{'C': 1000, 'penalty': 'l2'}


Let's see if a classifier with these parameters outperforms the standard model:

In [29]:
clsfr_std = LogReg(solver = 'liblinear')
clsfr_new = LogReg(solver = 'liblinear', C = 1000, penalty = "l2")

clsfr_std.fit(X_train, y_train)
clsfr_new.fit(X_train, y_train)

y_pred_std = clsfr_std.predict(X_test)
y_pred_new = clsfr_new.predict(X_test)

accuracy_report(y_test, y_pred_std)

Accuracy:  0.872
Recall:  0.849
Precision:  0.89
F1:  0.869


In [30]:
accuracy_report(y_test, y_pred_new)

Accuracy:  0.886
Recall:  0.873
Precision:  0.898
F1:  0.885


We managed to improve our classifier!

![](https://media.giphy.com/media/a0h7sAqON67nO/giphy.gif)

### Feature Assessment with [`eli5`](https://eli5.readthedocs.io/en/latest/index.html)

`eli5` is a great package to understand how our classifier makes decisions. It has two main functions: `show_weights()` tells us which features are most predictive for the classification, and `show_prediction()` explains us how each feature affects the prediction for a single example. This can be particularly useful for iterative feature selection and the exclusion of stopwords, etc.

In [32]:
!pip install eli5
import eli5
eli5.show_weights(clsfr, vec=vec, feature_names=vec.get_feature_names_out())

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107747 sha256=d1ac81b4da6efcdf709486e3b2481be83847c723a5d864a192b0daee31021b75
  Stored in directory: /root/.cache/pip/wheels/b8/58/ef/2cf4c306898c2338d51540e0922c8e0d6028e07007085c0004
Successfully built eli5


In [33]:
eli5.show_prediction(clsfr, reviews.text[400], vec=vec, feature_names=vec.get_feature_names_out())